# Get carbon intensity by country

We get our carbon intensity from [Our World In Data](https://ourworldindata.org/grapher/carbon-intensity-electricity).

>“Data Page: Carbon intensity of electricity generation”, part of the following publication: Hannah Ritchie, Pablo Rosado and Max Roser (2023) - “Energy”. Data adapted from Ember, Energy Institute. Retrieved from https://ourworldindata.org/grapher/carbon-intensity-electricity [online resource]

The data source and processing are available on [OWID Github](https://github.com/owid/energy-data).

We have to:
1. Filter for the last available year
1. Check if we not loose country we previously had.
1. Keep only the data we need
1. Export to JSON

## How to run this notebook?

```shell
python3 -m venv .venv-notebook-data
source .venv-notebook-data/bin/activate
pip install pandas numpy ipykernel
```

In [2]:
import pandas as pd
import numpy as np
import json

In [3]:
df = pd.read_csv("https://raw.githubusercontent.com/owid/energy-data/master/owid-energy-data.csv")
df.shape

(21812, 130)

In [4]:
df.query("country == 'India'")

,country,year,iso_code,population,gdp,biofuel_cons_change_pct,biofuel_cons_change_twh,biofuel_cons_per_capita,biofuel_consumption,biofuel_elec_per_capita,...,solar_share_elec,solar_share_energy,wind_cons_change_pct,wind_cons_change_twh,wind_consumption,wind_elec_per_capita,wind_electricity,wind_energy_per_capita,wind_share_elec,wind_share_energy
9205,India,1900,IND,2.807142e+08,2.716975e+11,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9206,India,1901,IND,2.818341e+08,2.773278e+11,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9207,India,1902,IND,2.831998e+08,3.006720e+11,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9208,India,1903,IND,2.848130e+08,3.047644e+11,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9209,India,1904,IND,2.864353e+08,3.060750e+11,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9324,India,2019,IND,1.383112e+09,9.595463e+12,23.248,2.451,9.396,12.996,22.782,...,2.852,1.309,4.979,7.327,166.895,45.774,63.31,120.666,3.903,1.791
9325,India,2020,IND,1.396387e+09,8.945314e+12,-6.260,-0.813,8.724,12.182,23.532,...,3.709,1.746,-4.560,-8.195,158.700,43.276,60.43,113.650,3.820,1.798
9326,India,2021,IND,1.407564e+09,9.801621e+12,61.522,7.495,13.979,19.677,26.116,...,3.984,1.865,12.688,19.481,178.181,48.374,68.09,126.588,3.971,1.859
9327,India,2022,IND,1.417173e+09,1.047625e+13,42.380,8.339,19.769,28.016,28.747,...,5.122,2.456,2.865,4.437,182.617,49.429,70.05,128.860,3.770,1.808


In [5]:
elec_col = [c for c in df.columns.to_list() if "_electricity" in c]
nuke_col = [c for c in elec_col if "nuclear" in c]
nuke_col

['nuclear_electricity']

In [6]:
carbon_intensity_col = [c for c in df.columns.to_list() if "intensity" in c]

In [7]:
latest_year = df['year'].max()
df[['country', 'iso_code', 'year'] + carbon_intensity_col + elec_col].query(f"iso_code=='FRA' and year=={latest_year}")

,country,iso_code,year,carbon_intensity_elec,biofuel_electricity,coal_electricity,fossil_electricity,gas_electricity,hydro_electricity,low_carbon_electricity,nuclear_electricity,oil_electricity,other_renewable_electricity,other_renewable_exc_biofuel_electricity,per_capita_electricity,renewables_electricity,solar_electricity,wind_electricity
7502,France,FRA,2023,56.039,9.5,2.16,43.3,31.43,53.19,470.81,335.65,9.71,10.1,0.6,7939.115,135.16,23.26,48.61


In [8]:
# keeping only relevant columns
dff = df[['country', 'iso_code', 'year'] + carbon_intensity_col + elec_col]
# getting rid of row with na carbon intensity for electricity
dff = dff.dropna(subset='carbon_intensity_elec')
# keeping the most recent available row
dff = dff.loc[dff.groupby('country')['year'].idxmax()]
# no empty isocode
df_filtered = dff[dff["iso_code"].notna()]
columns_names = {
    'country':'country_name',
    'carbon_intensity_elec':'carbon_intensity',
    'biofuel_electricity':'biofuel_TWh',
    'coal_electricity':'coal_TWh',
    'fossil_electricity':'fossil_TWh',
    'gas_electricity':'gas_TWh',
    'hydro_electricity':'hydroelectricity_TWh',
    'low_carbon_electricity':'low_carbon_TWh',
    'nuclear_electricity':'nuclear_TWh',
    'oil_electricity':'oil_TWh',
    'other_renewable_electricity':'other_renewable_TWh',
    'other_renewable_exc_biofuel_electricity':'other_renewable_exc_biofuel_TWh',
    'per_capita_electricity':'per_capita_Wh',
    'renewables_electricity':'renewables_TWh',
    'solar_electricity':'solar_TWh',
    'wind_electricity':'wind_TWh'
}
df_filtered.rename(columns=columns_names, inplace=True)
df_filtered = df_filtered[['country_name',
                        'iso_code',
                        'year',
                        'carbon_intensity',
                        'fossil_TWh',
                        'renewables_TWh',
                        'per_capita_Wh',
                        'coal_TWh',
                        'gas_TWh',
                        'oil_TWh',
                        'solar_TWh',
                        'wind_TWh',
                        'hydroelectricity_TWh',
                        'other_renewable_TWh',
                        'other_renewable_exc_biofuel_TWh',
                        'nuclear_TWh',
                        'biofuel_TWh',
                        'low_carbon_TWh']]
df_filtered.head(5)

/tmp/ipykernel_178936/160094593.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.rename(columns=columns_names, inplace=True)


,country_name,iso_code,year,carbon_intensity,fossil_TWh,renewables_TWh,per_capita_Wh,coal_TWh,gas_TWh,oil_TWh,solar_TWh,wind_TWh,hydroelectricity_TWh,other_renewable_TWh,other_renewable_exc_biofuel_TWh,nuclear_TWh,biofuel_TWh,low_carbon_TWh
146,Afghanistan,AFG,2022,132.530,0.13,0.70,20.181,0.03,0.00,0.10,0.08,0.00,0.62,0.00,0.0,0.0,0.00,0.70
635,Albania,ALB,2022,24.286,0.00,7.00,2462.779,0.00,0.00,0.00,0.04,0.00,6.96,0.00,0.0,0.0,0.00,7.00
758,Algeria,DZA,2022,634.611,87.50,0.68,1963.779,0.00,87.12,0.38,0.66,0.01,0.01,0.00,0.0,0.0,0.00,0.68
802,American Samoa,ASM,2022,611.111,0.17,0.01,4063.664,0.00,0.00,0.17,0.01,0.00,0.00,0.00,0.0,0.0,0.00,0.01
925,Angola,AGO,2022,174.734,4.23,12.71,475.990,0.00,1.67,2.56,0.02,0.00,12.64,0.05,0.0,0.0,0.05,12.71


In [9]:
columns_to_sum = ["fossil_TWh", "renewables_TWh", "nuclear_TWh"]
df_filtered['total_TWh'] = df_filtered[columns_to_sum].sum(axis=1)
df_filtered.head(2)

,country_name,iso_code,year,carbon_intensity,fossil_TWh,renewables_TWh,per_capita_Wh,coal_TWh,gas_TWh,oil_TWh,solar_TWh,wind_TWh,hydroelectricity_TWh,other_renewable_TWh,other_renewable_exc_biofuel_TWh,nuclear_TWh,biofuel_TWh,low_carbon_TWh,total_TWh
146,Afghanistan,AFG,2022,132.530,0.13,0.7,20.181,0.03,0.0,0.1,0.08,0.0,0.62,0.0,0.0,0.0,0.0,0.7,0.83
635,Albania,ALB,2022,24.286,0.00,7.0,2462.779,0.00,0.0,0.0,0.04,0.0,6.96,0.0,0.0,0.0,0.0,7.0,7.00


In [ ]:
df_filtered.query("iso_code=='ARE'")

,country_name,iso_code,year,carbon_intensity,fossil_TWh,renewables_TWh,per_capita_Wh,coal_TWh,gas_TWh,oil_TWh,solar_TWh,wind_TWh,hydroelectricity_TWh,other_renewable_TWh,other_renewable_exc_biofuel_TWh,nuclear_TWh,biofuel_TWh,low_carbon_TWh,total_TWh
7502,France,FRA,2023,56.039,43.3,135.16,7939.115,2.16,31.43,9.71,23.26,48.61,53.19,10.1,0.6,335.65,9.5,470.81,514.11


In [13]:
df_filtered.query("iso_code=='FRA'")

,country_name,iso_code,year,carbon_intensity,fossil_TWh,renewables_TWh,per_capita_Wh,coal_TWh,gas_TWh,oil_TWh,solar_TWh,wind_TWh,hydroelectricity_TWh,other_renewable_TWh,other_renewable_exc_biofuel_TWh,nuclear_TWh,biofuel_TWh,low_carbon_TWh,total_TWh
7502,France,FRA,2023,56.039,43.3,135.16,7939.115,2.16,31.43,9.71,23.26,48.61,53.19,10.1,0.6,335.65,9.5,470.81,514.11


renewables_Twh = hydroelectricity_Twh + wind_Twh + solar_Twh + other_renewable_Twh

RTE 2022 Production

![RTE](2023-07-07-22-40-48.png)



In [ ]:
# French electricity consumption in 2022 from https://ourworldindata.org/grapher/elec-mix-bar?stackMode=absolute&facet=metric&country=~FRA
# 57 TWh fossil + 298 TWh nuclear + 116 TWh renewable = 471 TWh
# From https://bilan-electrique-2021.rte-france.com/# we read 522 TWh for Production, but 468 TWh of consumption.
# Here we found 470 TWh, which is pretty close.

# According to https://analysesetdonnees.rte-france.com/bilan-electrique-2023/synthese
# For 2023 in France total production is 494,7 TWh (320,4 TWh nuclear, 50.8 TWh wind, 21.6 TWh solar, 58.8 TWh hydro, 30 TWh gaz) with 50,1 TWh exported
# The numbers is different from the one we have here : 514 TWh vs 494 TWh
# 2023 electricity consumption in France is 438,7 TWh according to RTE
# 2023 electricity carbon intensity in France is 32 gCO2eq per kilowatt-hour for 2023 according to RTE (5% of country emissions), versus 56 gCO2eq per kilowatt-hour in OurWorldInData !
# It will be nice if someone could explain the difference between the two sources. Maybe the difference is due to the overseas territories of France ?

df_filtered.query("iso_code=='FRA'")["total_TWh"].values[0]

np.float64(514.1099999999999)

In [16]:
df_filtered

,country_name,iso_code,year,carbon_intensity,fossil_TWh,renewables_TWh,per_capita_Wh,coal_TWh,gas_TWh,oil_TWh,solar_TWh,wind_TWh,hydroelectricity_TWh,other_renewable_TWh,other_renewable_exc_biofuel_TWh,nuclear_TWh,biofuel_TWh,low_carbon_TWh,total_TWh
146,Afghanistan,AFG,2022,132.530,0.13,0.70,20.181,0.03,0.00,0.10,0.08,0.00,0.62,0.000,0.0,0.0,0.00,0.70,0.83
635,Albania,ALB,2022,24.286,0.00,7.00,2462.779,0.00,0.00,0.00,0.04,0.00,6.96,0.000,0.0,0.0,0.00,7.00,7.00
758,Algeria,DZA,2022,634.611,87.50,0.68,1963.779,0.00,87.12,0.38,0.66,0.01,0.01,0.000,0.0,0.0,0.00,0.68,88.18
802,American Samoa,ASM,2022,611.111,0.17,0.01,4063.664,0.00,0.00,0.17,0.01,0.00,0.00,0.000,0.0,0.0,0.00,0.01,0.18
925,Angola,AGO,2022,174.734,4.23,12.71,475.990,0.00,1.67,2.56,0.02,0.00,12.64,0.050,0.0,0.0,0.05,12.71,16.94
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21112,Vietnam,VNM,2023,475.449,153.14,112.63,2688.376,124.31,26.52,2.31,25.46,10.02,76.79,0.853,NaN,0.0,0.36,112.63,265.77
21291,Western Sahara,ESH,2009,666.667,0.09,0.00,226.177,0.00,0.00,0.09,0.00,0.00,0.00,0.000,0.0,0.0,0.00,0.00,0.09
21470,Yemen,YEM,2022,566.102,2.35,0.60,87.546,0.00,0.54,1.81,0.60,0.00,0.00,0.000,0.0,0.0,0.00,0.60,2.95
21686,Zambia,ZMB,2022,111.967,2.16,17.31,972.641,2.16,0.00,0.00,0.14,0.00,17.09,0.080,0.0,0.0,0.08,17.31,19.47


In [17]:
energy_mix = {}
missings = []
for code in df_filtered['iso_code'].unique():
#for code in ['FRA', 'AFG', 'VNM']:
    # print(code)
    df=df_filtered.query("iso_code == @code")
    max_year = df.year.max()
    #if df.isnull().values.any() == False:
    if len(df)>0 and not np.isnan(df.loc[df.year == max_year].iloc[0]["carbon_intensity"]):
        energy_mix[code]=df.loc[df.year == max_year].iloc[0].replace(np.nan, 0).to_dict()
    else:
        missings.append(code)
print(f"Missing carbon_intensity for {len(missings)} countries over {len(df_filtered)}.")

Missing carbon_intensity for 0 countries over 213.


In [18]:
energy_mix.keys()

dict_keys(['AFG', 'ALB', 'DZA', 'ASM', 'AGO', 'ATG', 'ARG', 'ARM', 'ABW', 'AUS', 'AUT', 'AZE', 'BHS', 'BHR', 'BGD', 'BRB', 'BLR', 'BEL', 'BLZ', 'BEN', 'BMU', 'BTN', 'BOL', 'BIH', 'BWA', 'BRA', 'VGB', 'BRN', 'BGR', 'BFA', 'BDI', 'KHM', 'CMR', 'CAN', 'CPV', 'CYM', 'CAF', 'TCD', 'CHL', 'CHN', 'COL', 'COM', 'COG', 'COK', 'CRI', 'CIV', 'HRV', 'CUB', 'CYP', 'CZE', 'COD', 'DNK', 'DJI', 'DMA', 'DOM', 'TLS', 'ECU', 'EGY', 'SLV', 'GNQ', 'ERI', 'EST', 'SWZ', 'ETH', 'FLK', 'FRO', 'FJI', 'FIN', 'FRA', 'GUF', 'PYF', 'GAB', 'GMB', 'GEO', 'DEU', 'GHA', 'GIB', 'GRC', 'GRL', 'GRD', 'GLP', 'GUM', 'GTM', 'GIN', 'GNB', 'GUY', 'HTI', 'HND', 'HKG', 'HUN', 'ISL', 'IND', 'IDN', 'IRN', 'IRQ', 'IRL', 'ISR', 'ITA', 'JAM', 'JPN', 'JOR', 'KAZ', 'KEN', 'KIR', 'KWT', 'KGZ', 'LAO', 'LVA', 'LBN', 'LSO', 'LBR', 'LBY', 'LTU', 'LUX', 'MAC', 'MDG', 'MWI', 'MYS', 'MDV', 'MLI', 'MLT', 'MTQ', 'MRT', 'MUS', 'MEX', 'MDA', 'MNG', 'MNE', 'MSR', 'MAR', 'MOZ', 'MMR', 'NAM', 'NRU', 'NPL', 'NLD', 'NCL', 'NZL', 'NIC', 'NER', 'NGA', 'P

In [19]:
with open("global_energy_mix.json", "w") as outfile:
    json.dump(energy_mix, outfile, indent=4, sort_keys=True)

In [21]:
len(energy_mix.keys())
# 213 countries with keeping also old data

213

In [22]:
df_filtered[df_filtered["year"]<2022]

,country_name,iso_code,year,carbon_intensity,fossil_TWh,renewables_TWh,per_capita_Wh,coal_TWh,gas_TWh,oil_TWh,solar_TWh,wind_TWh,hydroelectricity_TWh,other_renewable_TWh,other_renewable_exc_biofuel_TWh,nuclear_TWh,biofuel_TWh,low_carbon_TWh,total_TWh
7544,French Guiana,GUF,2021,217.822,0.31,0.70,3395.392,NaN,NaN,0.31,0.06,NaN,0.59,NaN,NaN,NaN,0.05,0.70,1.01
8456,Guadeloupe,GLP,2021,500.000,1.10,0.56,4191.316,0.23,NaN,0.87,0.11,0.11,0.01,0.33,0.08,NaN,0.25,0.56,1.66
11798,Martinique,MTQ,2021,523.179,1.12,0.39,4094.371,NaN,NaN,1.12,0.09,0.05,NaN,0.25,0.00,NaN,0.25,0.39,1.51
16553,Reunion,REU,2021,572.816,2.22,0.87,3198.321,0.93,NaN,1.29,0.27,0.00,0.36,NaN,NaN,NaN,0.24,0.87,3.09
21291,Western Sahara,ESH,2009,666.667,0.09,0.00,226.177,0.00,0.0,0.09,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.09


In [80]:
df_filtered[df_filtered["year"]<2023]

,country_name,iso_code,year,carbon_intensity,fossil_TWh,renewables_TWh,per_capita_Wh,coal_TWh,gas_TWh,oil_TWh,solar_TWh,wind_TWh,hydroelectricity_TWh,other_renewable_TWh,other_renewable_exc_biofuel_TWh,nuclear_TWh,biofuel_TWh,low_carbon_TWh,total_TWh
146,Afghanistan,AFG,2022,132.530,0.13,0.70,20.181,0.03,0.00,0.10,0.08,0.00,0.62,0.00,0.0,0.0,0.00,0.70,0.83
635,Albania,ALB,2022,24.286,0.00,7.00,2462.779,0.00,0.00,0.00,0.04,0.00,6.96,0.00,0.0,0.0,0.00,7.00,7.00
758,Algeria,DZA,2022,634.611,87.50,0.68,1963.779,0.00,87.12,0.38,0.66,0.01,0.01,0.00,0.0,0.0,0.00,0.68,88.18
802,American Samoa,ASM,2022,611.111,0.17,0.01,4063.664,0.00,0.00,0.17,0.01,0.00,0.00,0.00,0.0,0.0,0.00,0.01,0.18
925,Angola,AGO,2022,174.734,4.23,12.71,475.990,0.00,1.67,2.56,0.02,0.00,12.64,0.05,0.0,0.0,0.05,12.71,16.94
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20987,Venezuela,VEN,2022,185.802,18.88,65.78,2991.340,0.00,13.25,5.63,0.01,0.09,65.68,0.00,0.0,0.0,0.00,65.78,84.66
21291,Western Sahara,ESH,2009,666.667,0.09,0.00,226.177,0.00,0.00,0.09,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.09
21470,Yemen,YEM,2022,566.102,2.35,0.60,87.546,0.00,0.54,1.81,0.60,0.00,0.00,0.00,0.0,0.0,0.00,0.60,2.95
21686,Zambia,ZMB,2022,111.967,2.16,17.31,972.641,2.16,0.00,0.00,0.14,0.00,17.09,0.08,0.0,0.0,0.08,17.31,19.47


In [24]:
df_filtered.query("iso_code=='DEU'")

,country_name,iso_code,year,carbon_intensity,fossil_TWh,renewables_TWh,per_capita_Wh,coal_TWh,gas_TWh,oil_TWh,solar_TWh,wind_TWh,hydroelectricity_TWh,other_renewable_TWh,other_renewable_exc_biofuel_TWh,nuclear_TWh,biofuel_TWh,low_carbon_TWh,total_TWh
7958,Germany,DEU,2023,380.95,231.48,264.56,6060.295,135.35,76.0,20.13,61.56,137.29,19.48,46.23,0.21,8.75,46.02,273.31,504.79
